In [18]:
import geopandas as gpd
import numpy as np
import sys

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

sys.path.append("/home/nilscp/GIT/")
from rastertools import utils 

In [19]:
lonlat_crs = ('GEOGCRS["Moon 2000",DATUM["D_Moon_2000",'
          'ELLIPSOID["Moon_2000_IAU_IAG",1737400,0,'
          'LENGTHUNIT["metre",1,ID["EPSG",9001]]]],'
          'PRIMEM["Greenwich",0,'
          'ANGLEUNIT["Decimal_Degree",0.0174532925199433]],'
          'CS[ellipsoidal,2],AXIS["longitude",east,ORDER[1],'
          'ANGLEUNIT["Decimal_Degree",0.0174532925199433]],'
          'AXIS["latitude",north,ORDER[2],'
          'ANGLEUNIT["Decimal_Degree",0.0174532925199433]]]')

eqc = utils.get_raster_crs("/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt").to_wkt()

In [20]:
eqc

'PROJCS["SIMPLE_CYLINDRICAL MOON",GEOGCS["GCS_MOON",DATUM["D_MOON",SPHEROID["MOON",1737400,0]],PRIMEM["Reference_Meridian",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Equirectangular"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'

In [3]:
lon = np.array([-52.38, -36.46, -107.47, -40.33, -10.47, -60.65, -34.28, 17.23, -26.11, -124.01, -6.79, -7.01, -14.71, -44.69, -50.47, -157.29, -82.83, 49.79, 7.79])
lat = np.array([-3.47, -12.84, 37.90, 0.94, -14.59, -40.64, 28.50, -33.72, -25.76, -17.08, 59.91, -54.34, 45.98, -12.43, -0.27, 18.30, 21.67, 15.72, 0.47])
diam = np.array([3703, 3053, 3208, 2950, 2072, 2069, 2120, 1737, 1875, 1215, 1077, 892, 805, 859, 819, 786, 687, 545, 643])
CRATER_ID = np.array(["crater_Flamsteed_S", "crater_Herigonius_K", "crater_high_0001", "crater_Encke_X", "crater_Lassell_D", "crater_high_0002", "crater_Samir", "crater_high_0003", "crater_mare_0001", "crater_high_0004", "crater_mare_0002", "crater_high_0005", "crater_mare_0003", "crater_high_0006", "crater_mare_0004", "crater_high_0007", "crater_high_0008", "crater_mare_0005", "crater_high_0009"])

In [33]:
gdf = gpd.GeoDataFrame(np.column_stack((lon,lat,diam,CRATER_ID)), columns=["lon","lat", "diam", "CRATER_ID"],geometry=gpd.points_from_xy(lon,lat,crs=lonlat_crs))

In [34]:
gdf.dtypes

lon            object
lat            object
diam           object
CRATER_ID      object
geometry     geometry
dtype: object

In [35]:
gdf = gdf.astype({'lon': 'float32', 'lat': 'float32', 'diam': 'float32'})

In [36]:
gdf.head(5)

,lon,lat,diam,CRATER_ID,geometry
0,-52.380001,-3.470000,3703.0,crater_Flamsteed_S,POINT (-52.38000 -3.47000)
1,-36.459999,-12.840000,3053.0,crater_Herigonius_K,POINT (-36.46000 -12.84000)
2,-107.470001,37.900002,3208.0,crater_high_0001,POINT (-107.47000 37.90000)
3,-40.330002,0.940000,2950.0,crater_Encke_X,POINT (-40.33000 0.94000)
4,-10.470000,-14.590000,2072.0,crater_Lassell_D,POINT (-10.47000 -14.59000)


In [37]:
gdf_eqc = gdf.to_crs(eqc) 

In [38]:
gdf_eqc.head(5)

,lon,lat,diam,CRATER_ID,geometry
0,-52.380001,-3.470000,3703.0,crater_Flamsteed_S,POINT (-1588337.095 -105222.026)
1,-36.459999,-12.840000,3053.0,crater_Herigonius_K,POINT (-1105589.356 -389351.819)
2,-107.470001,37.900002,3208.0,crater_high_0001,POINT (-3258850.470 1149254.981)
3,-40.330002,0.940000,2950.0,crater_Encke_X,POINT (-1222940.723 28503.949)
4,-10.470000,-14.590000,2072.0,crater_Lassell_D,POINT (-317485.479 -442417.683)


In [39]:
gdf_eqc.to_file("/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc.shp")

## Clip DEM and visible image for coldspots (SLDEM2013)

In [64]:
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_kaguya_manually_checked.shp"
dem = "/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 8.0
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2013/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/validation/TCOMAP/"
identifier_dem = "Kaguya_SLDEM2013"
identifier_orthoimage = "Kaguya_TCOMAP_02"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2013/"

In [65]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 19/19 [01:06<00:00,  3.48s/it]


In [66]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 19/19 [00:52<00:00,  2.77s/it]


In [67]:
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_manually_checked.shp"
dem = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LrocKaguya_DEMmerge_60N60S_512ppd.tif"
orthoimage = "/home/nilscp/QGIS/Moon/globalMosaics/Lunar_LRO_LROC-WAC_Mosaic_global_100m_June2013.tif"
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2015/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/validation/WAC/"
identifier_dem = "SLDEM2015"
identifier_orthoimage = "WAC"

In [68]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 19/19 [00:23<00:00,  1.25s/it]


In [69]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 19/19 [00:21<00:00,  1.16s/it]


## Generate regional (2.0-3.0R) detrended DEM (SLDEM2013, Coldspots craters)

In [70]:
scaling_factor = 1.0
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_kaguya_manually_checked.shp"
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2013/"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2013/"
dem_detrended_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2013_detrended/"
craterID = None
overwrite=True

In [71]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 19/19 [00:24<00:00,  1.28s/it]


In [72]:
scaling_factor = 0.5
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_manually_checked.shp"
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2015/"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2015/"
dem_detrended_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2015_detrended/"
craterID = None
overwrite=True

In [73]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 19/19 [00:08<00:00,  2.30it/s]


## Detect rim composite, and update the centre of the crater (Coldspots)

In [52]:
#scaling_factor = 1.0
suffix = '_SLDEM2015_detrended.tif'

In [53]:
rim_detection.main(location_of_craters, dem_detrended_folder, shp_folder, suffix, threshold_min=None, threshold_max=None , craterID = None)

  0%|          | 0/19 [00:00<?, ?it/s]/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:1033: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename)
100%|██████████| 19/19 [01:29<00:00,  4.72s/it]


In [60]:
suffix = '_Kaguya_SLDEM2013_detrended.tif'
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2013/"
dem_detrended_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2013_detrended/"

In [61]:
rim_detection.main(location_of_craters, dem_detrended_folder, shp_folder, suffix, threshold_min=None, threshold_max=None , craterID = None)

  0%|          | 0/19 [00:00<?, ?it/s]/home/nilscp/GIT/crater_morphometry/rim_detection/rim_detection.py:1033: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file(filename)
100%|██████████| 19/19 [04:37<00:00, 14.60s/it]


## Manual check of results
It is important to manually check if the detected rims give meaningful results. If not, the polygon can be deleted, and replace by a manually generated ellipse. This should (hopefully) not happen in many cases for fresh impact structures (<5%). However, this is likely to happen more often for degraded craters (such as the rayed crater dataset, that contains both young and old craters). For completion, we are writing down the id of the crater where a problem is detected, to understand if we can improve the rule-based detection of rim in the future. 

## Update centre of craters
- Merge the centres of craters together
- Merge the polygons of rim composite together (for a faster inspection of results). I think I will here 
- If this step is not working properly for a crater, its id is written down (most likely due to too low resolution of the DEM, or very special conditions)

In [10]:
out_shapefile = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_manually_checked.shp"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2015/polygons/"
old_crater_centres = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc.shp"

In [ ]:
out_shapefile = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_kaguya_manually_checked.shp"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2013/polygons/"
old_crater_centres = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_kaguya.shp"

I have a few issues with the indexing of the pandas and geopandas dataframe.. I have run the line below within python to fix index issues...

In [11]:
rim_detection.update_crater_centres(shp_folder, out_shapefile, old_crater_centres, replace_crs=False)

100%|██████████| 386/386 [04:20<00:00,  1.48it/s]


KeyError: '[405, 428, 432, 444, 582, 585, 638, 644, 664, 689, 700, 777, 856, 893, 905, 907, 919, 960, 962, 992, 1010, 1012, 1016, 1021, 1034, 1049, 1050, 1056, 1089, 1117, 1138, 1141, 1189, 1193, 1235, 1248, 1322, 1328, 1339, 1342, 1348, 1350, 1352, 1357, 1358, 1374, 1381, 1390, 1438, 1449, 1452, 1453, 1476, 1512, 1524, 1525, 1535, 1537, 1538, 1541, 1553, 1574, 1579, 1580, 1581, 1592, 1595, 1599, 1606, 1608, 1616, 1617, 1621, 1627, 1634, 1635, 1638, 1649, 1658, 1661, 1662, 1663, 1668, 1670, 1672, 1675, 1679, 1680, 1686, 1688, 1697, 1701, 1705, 1722, 1724, 1725, 1726, 1727, 1730, 1733, 1734, 1736, 1754, 1760, 1763, 1767, 1770, 1774, 1777, 1778, 1782, 1783, 1801, 1803, 1810, 1811, 1821, 1823, 1829, 1830, 1833, 1840, 1843, 1852, 1853, 1856, 1859, 1863, 1865, 1866, 1867, 1872, 1873, 1877, 1878, 1884, 1886, 1887, 1889, 1891, 1892, 1893, 1901, 1903, 1912, 1916, 1932, 1935, 1936, 1938, 1941, 1943, 1944, 1945, 1946, 1952, 1962, 1968, 1970, 1988, 1999, 2016, 2056, 2058, 2071, 2075, 2077, 2086, 

## Update the DEM, and the 2.0R-3.0R and 0.9R-1.1R detrendings
To have the right DEMs

In [13]:
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/ColdSpots/final_crater_center/global/coldspots_larger_than_250m_EQC_manually_checked.shp"
dem = "/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 8.0
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_ColdSpots/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/TCOMAP_ColdSpots/"
identifier_dem = "Kaguya_SLDEM2013"
identifier_orthoimage = "Kaguya_TCOMAP_02"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/ColdSpots/final_crater_centre/"

In [14]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 386/386 [15:00<00:00,  2.33s/it]


In [15]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 386/386 [12:46<00:00,  1.98s/it]


In [16]:
scaling_factor = 1.0
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_ColdSpots/"
dem_detrended_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_ColdSpots_detrended/"
craterID = None
overwrite=True

In [17]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=craterID, overwrite=overwrite)

100%|██████████| 386/386 [03:34<00:00,  1.80it/s]


## Calculate morphometrical parameters from the second ellipse (of the second run)
- load ellipse in pixel coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r
- I could add the possibility to add manually an ellipse for the few cases where the ellipse is not representing well the rim of the crater

In [ ]:
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_manually_checked.shp"
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2015/polygons'
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2015_detrended/"
suffix = '_SLDEM2015_detrended.tif'
prefix = 'crater'
craterID=None

In [ ]:
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/validation/crater_centres/validation_eqc_kaguya_manually_checked.shp"
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/validation/shapefiles/SLDEM2013/polygons'
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/validation/SLDEM2013_detrended/"
suffix = '_Kaguya_SLDEM2013_detrended.tif'
prefix = 'crater'
craterID=None